In [5]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.4
numpy 1.19.5
pandas 1.1.5
sklearn 0.24.2
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


# constant是常量张量

In [ ]:
# constant是常量张量,类似于tuple不可变
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])

# index
#2.0能够直接获取值时因为execution默认打开的
print(t)
print('-' * 50)
print(t[:, 1:])
print('-' * 50)
# 这里的'...'和':'一样功能
print(t[..., 1])
# t.assign(1)对常量不能进行再次assign设置
print('-' * 50)
#转为ndarray
type(t.numpy())
q = t.numpy()

In [ ]:
t1 = tf.constant(q)  #把ndarray变为张量
t1

In [ ]:
# ops 使用tf本身的math接口对Tensor进行计算
print(t + 10)
print('-' * 50)
print(t)
print('-' * 50)
print(tf.square(t))
print('-' * 50)
print(t)
#矩阵乘以自己的转置
print('-' * 50)
# transpose是矩阵的转置
print(tf.transpose(t))
print('-' * 50)
# @是矩阵乘法（线代里的矩阵乘法），和*不一致
print(t @ tf.transpose(t))

In [ ]:
print(tf.sqrt(t))
print('-' * 50)
# tf.math.sqrt(t)
tf.math.log(t)  #必须加math

## numpy conversion

In [ ]:
print(t.numpy())  #可以直接通过numpy取出来
print(t.numpy().tolist())
print(type(t.numpy()))
print(np.square(t))  #直接求平方
np_t = np.array([[1., 2., 3.], [4., 5., 6.]])
print(tf.constant(np_t))  #转换为tensor

## Scalars  就是标量，只有一个数值的张量，称为标量

In [ ]:
t = tf.constant(2.718)
print(t)
print(t.numpy())
print(t.shape)  #维数

In [ ]:
# strings
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

## string array

In [ ]:
t = tf.constant(["cafe", "coffee", "咖啡"])
#自动求出数组中每一个字符的长度,如果不加unit="UTF8_CHAR"，得到的是实际字节存储的长度(即一个汉字3B)
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.length(t, unit="BYTE"))
r = tf.strings.unicode_decode(t, "UTF8")
# https://tool.chinaz.com/tools/unicode.aspx  汉字转的是unicode编码
print(r)
# RaggedTensor 是指形状分布不固定的（行元素个数不相等）
# Tensor，2.0新增

# ragged tensor：每一行的元素个数不相等，Numpy和NDArray是不可以这样的

In [ ]:
r = tf.ragged.constant([[11, 12], [21, 22, 23], [], [41]])

# index op
print(r)
print('-' * 50)
print(r.shape)
print('-' * 50)
# 取一行是tensor
print(r[1])
#取一行也是ragged tensor
print('-' * 50)
print(r[1:3])
# print(r[:,1])#不能取列索引，只能取行索引，之前shape的列变量都是None

In [ ]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51, 52], [], [], [71]])
print(tf.concat([r, r2], axis=0))
print(tf.concat([r, r2], axis=1))  #行数不相等，不可以拼
#是否可以把上面的axis改为1

In [ ]:
#按轴1进行合并时，行数要一致，行数相等，可以拼
r3 = tf.ragged.constant([[13, 14], [15], [41], [42, 43]])
print(tf.concat([r, r3], axis=1))

In [ ]:
print(r.to_tensor())  #各种深度学习模型必须输入一个tensor
#空闲的补0，只能往后面补

# sparse tensor 可以往前面补零,sparse tensor从第一行依次往下填位置

In [ ]:
# sparese tensor存储节省内存空间，磁盘空间
s = tf.SparseTensor(indices=[[0, 1], [1, 0], [2, 3], [3, 2]],  #位置，和下面的value配合，在相应的位置赋相对应的值
                    values=[1., 2., 3., 5],  #值
                    dense_shape=[4, 4])  #维数
print(s)
tt = tf.sparse.to_dense(s)
tt

In [ ]:
# ops on sparse tensors
# 可以看到位置没变，value变了
s2 = s * 2.0
print(s2)

#不支持加法
try:
    s3 = s + 1
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10., 20.],
                  [30., 40.],
                  [50., 60.],
                  [70., 80.]])
# tf.sparse.to_dense(s)@s4，和下面代码等价
#稀疏Tensor和Tensor相乘，会自动将s变为sparse.to_dense(s)再相乘
print(tf.sparse.sparse_dense_matmul(s, s4))

In [ ]:
# sparse tensor
s5 = tf.SparseTensor(indices=[[0, 2], [2, 3], [0, 1]],
                     values=[1., 2., 3.],
                     dense_shape=[3, 4])
# print(tf.sparse.to_dense(s5))
# #sparse无顺序时，不能转为tensor，会报错
# (是索引顺序，这个例子中的indices的顺序，因为to_dense是一行一行的变）
print(s5)
# 没有顺序的st必须进行reorder以后才能to_dense
s6 = tf.sparse.reorder(s5)
print(s6)
print(tf.sparse.to_dense(s6))

# Variables

In [ ]:
# Variables，可以变
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
print(v.value())
print('-' * 50)
print(v.numpy())

In [ ]:
# 修改变量时要用assign，改变tensor内某个值，空间没有发生变化，效率高
# assign value
print(id(v))
v.assign(2 * v)  # 矩阵乘一个数
print(id(v))
print(v.numpy())
print('-' * 50)
v[0, 1].assign(42)  #取某个元素修改
print(v.numpy())
print('-' * 50)
v[1].assign([7., 8., 9.])  #取某一行修改
print(v.numpy())
print(id(v))

In [ ]:
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

In [ ]:
# 没有用assign
v = 2 * v
print(v)
print(id(v))
print(type(v))

In [ ]:
x = tf.constant([[1., 1.], [2., 2.]])
tf.reduce_mean(x, axis=1)